In [1]:
# Read libraries and files
import pandas as pd
import json
from urllib.request import urlopen
import os

In [2]:
# Real time dataset
url = 'https://api.data.gov.sg/v1/transport/carpark-availability'
fileobj = urlopen(url)
data = json.load(fileobj)["items"][0]
timestamp = data.get("timestamp")
carpark_data = data.get("carpark_data")
def clean_carpark(carpark):
    carpark_info = carpark.pop("carpark_info")[0]
    carpark_info.update(carpark)
    return carpark_info
carpark_data_cleaned = map(clean_carpark, carpark_data)
carpark_df = pd.DataFrame.from_dict(carpark_data_cleaned)
print(carpark_df)
    
    

     total_lots lot_type lots_available carpark_number      update_datetime
0           105        C             44           HE12  2023-02-27T09:37:52
1           583        C            208            HLM  2023-02-27T09:37:40
2           329        C            212            RHM  2023-02-27T09:37:52
3            97        C             21           BM29  2023-02-27T09:37:59
4            96        C              0            Q81  2023-02-27T09:37:34
...         ...      ...            ...            ...                  ...
1917        249        C            161            B8B  2023-02-27T09:37:51
1918         10        H              0            BM6  2023-02-27T09:21:53
1919        500        C            442            BA6  2023-02-27T09:35:54
1920        352        C            306            BA8  2023-02-27T09:35:47
1921        508        C            441            BA7  2023-02-27T09:35:47

[1922 rows x 5 columns]


In [3]:
# Static dataset on carpark information
carpark_details = pd.read_csv('Input/hdb-carpark-information.csv')
carpark_merged = pd.merge(carpark_df, carpark_details, how="left", left_on="carpark_number", right_on = "car_park_no").drop('car_park_no', axis=1)
print(carpark_merged)

     total_lots lot_type lots_available carpark_number      update_datetime  \
0           105        C             44           HE12  2023-02-27T09:37:52   
1           583        C            208            HLM  2023-02-27T09:37:40   
2           329        C            212            RHM  2023-02-27T09:37:52   
3            97        C             21           BM29  2023-02-27T09:37:59   
4            96        C              0            Q81  2023-02-27T09:37:34   
...         ...      ...            ...            ...                  ...   
1917        249        C            161            B8B  2023-02-27T09:37:51   
1918         10        H              0            BM6  2023-02-27T09:21:53   
1919        500        C            442            BA6  2023-02-27T09:35:54   
1920        352        C            306            BA8  2023-02-27T09:35:47   
1921        508        C            441            BA7  2023-02-27T09:35:47   

                          address     x_coord     y

In [4]:
# Obtaining latitude and longitude from oneAPI
def convert(x, y, choice):
    url = 'https://developers.onemap.sg/commonapi/convert/3414to4326?X=' + str(x) + '%20&' + 'Y=' + str(y)
    fileobj = urlopen(url)
    coordinates = json.load(fileobj)
    return coordinates.get(choice)
latitude = ['latitude']*len(carpark_merged)
longitude = ['longitude']*len(carpark_merged)
carpark_merged['latitude'] = list(map(convert, carpark_merged['x_coord'], carpark_merged['y_coord'], latitude))
carpark_merged['longitude'] = list(map(convert, carpark_merged['x_coord'], carpark_merged['y_coord'], longitude))


In [6]:
print(carpark_merged)

     total_lots lot_type lots_available carpark_number      update_datetime  \
0           105        C             44           HE12  2023-02-27T09:37:52   
1           583        C            208            HLM  2023-02-27T09:37:40   
2           329        C            212            RHM  2023-02-27T09:37:52   
3            97        C             21           BM29  2023-02-27T09:37:59   
4            96        C              0            Q81  2023-02-27T09:37:34   
...         ...      ...            ...            ...                  ...   
1917        249        C            161            B8B  2023-02-27T09:37:51   
1918         10        H              0            BM6  2023-02-27T09:21:53   
1919        500        C            442            BA6  2023-02-27T09:35:54   
1920        352        C            306            BA8  2023-02-27T09:35:47   
1921        508        C            441            BA7  2023-02-27T09:35:47   

                          address     x_coord     y

In [5]:
# Save combined dataset
outname = 'carpark_combined.csv'
outdir = './Output'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

carpark_merged.to_csv(fullname, index=False)